#### Tinhamos que a idade dos clientes e a idade possuiam uma forte correlação linear (em torno de 0.8 ), e a taxa do empréstimo possuia um forte correlação não linear com o grau do empréstimo ( em torno de  0.83 para o coeficiente kendall ), como temos coeficientes altos vamos utilizar essas duas informações para remover as outras variáveis que podem causar ruídos na nossa estimativa.

# tratando os dados faltantes da taxa do empréstimo

#### Vamos usar o grau do empréstimo que é uma variável categórica para prever os valores da taxa do empréstimo, o que é um dificultador para qualquer modelo, vamos testar pegar a média da taxa pelo grau do empréstimo. 

In [1]:
# manipular dados
import pandas as pd
import numpy as np

# ajustar modelos
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [2]:
df_com_na = pd.read_csv('../dados/credit_risk_dataset.csv')
df_com_na = df_com_na.drop(columns='Unnamed: 0', axis=1)

In [3]:
df = df_com_na.copy()

In [4]:
df_no_nan = df.dropna(subset=['loan_int_rate', 'person_emp_length'])

In [5]:
int_rate_p_grade = df_no_nan.groupby('loan_grade')['loan_int_rate'].mean().reset_index()
int_rate_p_grade

,loan_grade,loan_int_rate
0,A,7.343587
1,B,11.009722
2,C,13.458466
3,D,15.333632
4,E,16.978966
5,F,18.481064
6,G,20.250606


In [6]:
def substitute_loan_int_rate(row: np.array):
    '''
    Função para substituir dados faltantes do loan_int_rate
    baseado na média dos graus de empréstimos
    '''
    global int_rate_p_grade 
    print(row)
    if pd.isna(row['loan_int_rate']):

        grade = row['loan_grade']
        mean_int_rate = int_rate_p_grade.loc[int_rate_p_grade['loan_grade']==grade, 'loan_int_rate']
        row['loan_int_rate'] = mean_int_rate
        
    else:
        row = row
        
    return row

#### Vamos pegar só as rows com valores faltantes na taxa de empréstimo afim de tornar o processo de substituição mais eficiente

In [7]:
df_na = df[df['loan_int_rate'].isnull()]
df_na = df_na.apply(lambda x: substitute_loan_int_rate(x),axis=1)

person_age                           33
person_income                     66500
person_home_ownership          MORTGAGE
person_emp_length                   1.0
loan_intent                   EDUCATION
loan_grade                            A
loan_amnt                          9200
loan_int_rate                       NaN
loan_percent_income                0.14
cb_person_default_on_file             N
cb_person_cred_hist_length            6
loan_status                           0
Name: 16, dtype: object
person_age                                   25
person_income                             45996
person_home_ownership                      RENT
person_emp_length                           2.0
loan_intent                   DEBTCONSOLIDATION
loan_grade                                    G
loan_amnt                                  5000
loan_int_rate                               NaN
loan_percent_income                        0.11
cb_person_default_on_file                     Y
cb_person_cred_h

#### Agora vamos substituir as rows onde foi substituído o dado faltante da taxa de empréstimo no dataset original nos baseando no indice presente nos datasets

In [8]:
# vamos substituir do
df.loc[ df_na.index ] = df_na

C:\Users\Gabri\AppData\Local\Temp\ipykernel_13248\1786368221.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0    7.343587
 Name: loan_int_rate, dtype: float64 6    20.250606
                                     Name: loan_int_rate, dtype: float64
 1    11.009722
 Name: loan_int_rate, dtype: float64 ...
 3    15.333632
 Name: loan_int_rate, dtype: float64 0    7.343587
                                     Name: loan_int_rate, dtype: float64
 1    11.009722
 Name: loan_int_rate, dtype: float64]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[ df_na.index ] = df_na


In [9]:
df.isna().sum()

person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length             552
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                   0
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
loan_status                     0
dtype: int64

# Substituindo os dados do tempo de empregamento person_emp_length

#### Considerando que verificamos que a variável person_emp_length que representa o tempo de empregamento tinha uma forte correlação linear com a idade podemos ajustar uma regressão linear simples para estimar os dados faltantes.

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

X = pd.DataFrame(df_no_nan['person_age'])
y = pd.DataFrame(df_no_nan['person_emp_length'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

regressor = LinearRegression()  
regressor.fit(X_train, y_train) 

# To make predictions on the test data
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_true=y_test, y_pred=y_pred, squared=True)
rmse = mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False)
r_squared = r2_score(y_test, y_pred)

print( 'mse:', mse, 'rmse:', rmse, 'R-quadrado', r_squared )

mse: 15.904902489442296 rmse: 3.9880950953359044 R-quadrado 0.026584662358804745


c:\Users\Gabri\Desktop\PROJETOS\remake_risco_credito\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Gabri\Desktop\PROJETOS\remake_risco_credito\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Podemos notar que a regressão linear obteve um r quadrado baixo, mas um erro médio de 3.9, o que não é um erro grande, considerando que utilizamos a variável com maior correlação com o tempo de empregamento e que o erro médio não está grande vamos continuar com esse método.

In [11]:
regressor = LinearRegression()  
regressor.fit(X, y) 

LinearRegression()

In [20]:
# compute with statsmodels, by adding intercept manually
import statsmodels.api as sm
# X1 = sm.add_constant(X)
result = sm.OLS(y, X).fit()
#print dir(result)
print( result.rsquared, result.rsquared_adj)


0.5922673214866064 0.5922435580713766


In [ ]:
def substitute_na_person_emp_length( row ):

    global regressor 
    value = pd.DataFrame({'person_age': row['person_age']}, index=[0])
    # print(regressor.predict( value  )[0][0] )
    # .predict da regressao retorna lista com lista dentro
    # então é necessário pegar o numero com os indices
    row['person_emp_length'] = regressor.predict( value  )[0][0]
    
    return row

In [ ]:
df_na = df[df['person_emp_length'].isnull()]
df_na = df_na.apply(lambda x: substitute_na_person_emp_length(x),axis=1)

In [ ]:
df_na.isna().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64

In [ ]:
# vamos substituir do
df.loc[ df_na.index ] = df_na

In [ ]:
df.isna().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64

Vamos comparar aos valores anteriores com os valores faltantes olhando no notebook introdução

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19548 entries, 0 to 19547
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  19548 non-null  int64  
 1   person_income               19548 non-null  int64  
 2   person_home_ownership       19548 non-null  object 
 3   person_emp_length           19548 non-null  float64
 4   loan_intent                 19548 non-null  object 
 5   loan_grade                  19548 non-null  object 
 6   loan_amnt                   19548 non-null  int64  
 7   loan_int_rate               19548 non-null  object 
 8   loan_percent_income         19548 non-null  float64
 9   cb_person_default_on_file   19548 non-null  object 
 10  cb_person_cred_hist_length  19548 non-null  int64  
 11  loan_status                 19548 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


In [ ]:
df['loan_int_rate'] = df['loan_int_rate'].astype(float)

C:\Users\Gabri\AppData\Local\Temp\ipykernel_11176\141036047.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df['loan_int_rate'] = df['loan_int_rate'].astype(float)


In [ ]:
df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,19548.000000,1.954800e+04,19548.000000,19548.000000,19548.000000,19548.000000,19548.000000,19548.000000
mean,27.713833,6.617245e+04,4.779211,9595.420248,11.025592,0.169930,5.795478,0.218181
std,6.340725,6.558040e+04,3.980277,6320.414975,3.212845,0.106111,4.044860,0.413021
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.090000,3.000000,0.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,11.009722,0.150000,4.000000,0.000000
75%,30.000000,8.000000e+04,7.000000,12025.000000,13.458466,0.230000,8.000000,0.000000
max,144.000000,6.000000e+06,38.000000,35000.000000,22.480000,0.770000,30.000000,1.000000


In [ ]:
df_com_na.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,19548.000000,1.954800e+04,18996.000000,19548.000000,17665.000000,19548.000000,19548.000000,19548.000000
mean,27.713833,6.617245e+04,4.780217,9595.420248,11.025886,0.169930,5.795478,0.218181
std,6.340725,6.558040e+04,4.036093,6320.414975,3.231083,0.106111,4.044860,0.413021
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.090000,3.000000,0.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.150000,4.000000,0.000000
75%,30.000000,8.000000e+04,7.000000,12025.000000,13.480000,0.230000,8.000000,0.000000
max,144.000000,6.000000e+06,38.000000,35000.000000,22.480000,0.770000,30.000000,1.000000


#### Distribuição dos dados com os dados faltantes substituídos pelos métodos que utilizamos

In [ ]:
# separando os dados em inadimplentes enão inadimplentes 
inadimplentes = df.loc[df['loan_status']==1]
n_inadimplentes = df.loc[df['loan_status']==0]

In [ ]:
inadimplentes.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,4265.000000,4265.000000,4265.000000,4265.000000,4265.000000,4265.000000,4265.000000,4265.0
mean,27.445252,49678.865182,4.168248,10861.776084,13.000488,0.245341,5.703165,1.0
std,6.221209,37801.329057,3.834284,7179.303678,3.301165,0.131263,4.137534,0.0
min,20.000000,4000.000000,0.000000,1000.000000,5.420000,0.010000,2.000000,1.0
25%,23.000000,30000.000000,1.000000,5000.000000,10.750000,0.140000,3.000000,1.0
50%,26.000000,41682.000000,3.000000,9600.000000,13.458466,0.230000,4.000000,1.0
75%,30.000000,60000.000000,6.000000,15000.000000,15.333632,0.340000,8.000000,1.0
max,70.000000,604000.000000,34.000000,35000.000000,22.480000,0.770000,30.000000,1.0


In [ ]:
n_inadimplentes.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,15283.000000,1.528300e+04,15283.000000,15283.000000,15283.000000,15283.000000,15283.000000,15283.0
mean,27.788785,7.077529e+04,4.949712,9242.020546,10.474461,0.148885,5.821239,0.0
std,6.371859,7.074791e+04,4.003588,6011.873228,2.961463,0.086974,4.018375,0.0
min,20.000000,7.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.0
25%,23.000000,4.200000e+04,2.000000,5000.000000,7.660000,0.080000,3.000000,0.0
50%,26.000000,6.000000e+04,4.000000,8000.000000,10.650000,0.130000,4.000000,0.0
75%,30.000000,8.479000e+04,7.000000,12000.000000,12.690000,0.200000,8.000000,0.0
max,144.000000,6.000000e+06,38.000000,35000.000000,22.060000,0.700000,30.000000,0.0


distribuição dos dados antes sem nenhuma substituição dos dados faltantes

In [ ]:
# separando os dados em inadimplentes enão inadimplentes 
inadimplentes = df_com_na.loc[df_com_na['loan_status']==1]
n_inadimplentes = df_com_na.loc[df_com_na['loan_status']==0]

In [ ]:
inadimplentes.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,4265.000000,4265.000000,4093.000000,4265.000000,3849.000000,4265.000000,4265.000000,4265.0
mean,27.445252,49678.865182,4.139995,10861.776084,13.032741,0.245341,5.703165,1.0
std,6.221209,37801.329057,3.908621,7179.303678,3.301576,0.131263,4.137534,0.0
min,20.000000,4000.000000,0.000000,1000.000000,5.420000,0.010000,2.000000,1.0
25%,23.000000,30000.000000,1.000000,5000.000000,10.740000,0.140000,3.000000,1.0
50%,26.000000,41682.000000,3.000000,9600.000000,13.480000,0.230000,4.000000,1.0
75%,30.000000,60000.000000,6.000000,15000.000000,15.580000,0.340000,8.000000,1.0
max,70.000000,604000.000000,34.000000,35000.000000,22.480000,0.770000,30.000000,1.0


In [ ]:
n_inadimplentes.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,15283.000000,1.528300e+04,14903.000000,15283.000000,13816.000000,15283.000000,15283.000000,15283.0
mean,27.788785,7.077529e+04,4.956049,9242.020546,10.466796,0.148885,5.821239,0.0
std,6.371859,7.074791e+04,4.052867,6011.873228,2.979545,0.086974,4.018375,0.0
min,20.000000,7.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.0
25%,23.000000,4.200000e+04,2.000000,5000.000000,7.740000,0.080000,3.000000,0.0
50%,26.000000,6.000000e+04,4.000000,8000.000000,10.620000,0.130000,4.000000,0.0
75%,30.000000,8.479000e+04,7.000000,12000.000000,12.690000,0.200000,8.000000,0.0
max,144.000000,6.000000e+06,38.000000,35000.000000,22.060000,0.700000,30.000000,0.0


#### Podemos observar que tivemos mudanças mínimas na distribuição dos dados utilizando os métodos não tradicionais que utilizamos e que fazem mais sentido para esse caso do que técnicas genéricas.

In [ ]:
df.to_csv('../dados/df_sem_na.csv')